# Load DeepFly3D Data

In [27]:
%load_ext autoreload
%autoreload 2
from load import *
import torch
import yaml
import logging
from imp import reload
import matplotlib.pyplot as plt
from liftpose.vision_3d import world_to_camera_dict, reprojection_error, intrinsic_matrix
reload(logging)
logger = logging.getLogger(__name__).setLevel(logging.INFO)
from tqdm import tqdm
tqdm.get_lock().locks = []

# decleare data parameters
par_train = {  'data_dir'       : '/data/LiftPose3D/fly_tether/data_DF3D', # change the path 
               'out_dir'        : '/data/LiftPose3D/fly_tether/angle_inv_network',
               'train_subjects' : [1],
               'test_subjects'  : [6,7],
               'actions'        : ['all'],
               'cam_id'         : [0,6]}

# merge with training parameters
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}

# Load 2D data
train_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_2d = load_2D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)

# Load 3D data
train_3d, train_keypts, rcams_train = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["train_subjects"],
    actions=par["actions"],
)
test_3d, test_keypts, rcams_test = load_3D(
    par["data_dir"],
    par,
    cam_id=par["cam_id"],
    subjects=par["test_subjects"],
    actions=par["actions"],
)

test_3d = world_to_camera_dict(test_3d, rcams_test)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Train LiftPose3D Network on DeepFly3D Data

In [16]:
from liftpose.main import train as lp3d_train
from liftpose.lifter.augmentation import random_project
from liftpose.preprocess import obtain_projected_stats,  get_visible_points

intr = intrinsic_matrix(171.0 * 94.0, 171.0 * 94.0, 240, 480)
#narrow
training_kwargs = {'eangles': {0: [[-30,-20], [-2, 2], [-2,2]], 1: [[-270,-250], [-2, 2], [-2,2]]},
                   'axsorder': 'yxz', 
                   'intr': intr}
#wide
#training_kwargs = {'eangles': {0: [[-90,-10], [-10, 10], [-2,2]], 1: [[-270,-180], [-10, 10], [-2,2]]},
#                   'axsorder': 'yxz', 
#                   'intr': intr}

aug = random_project(**training_kwargs)
d_train = get_visible_points(train_3d, train_keypts)
stats = obtain_projected_stats(d_train, 
                              training_kwargs['eangles'], 
                              training_kwargs['axsorder'], 
                              training_kwargs['intr'], 
                              par['roots'], 
                              par['target_sets'],
                              par['out_dir'], 
                              th=0.05)

[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 1457.3895170879246
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 15.415593097536046
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 9.926375088710847
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 17.15570434688674
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 6.5767270201804875
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 3.979429595523893
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 1.8888398217881415
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 4.374027288673904
[preprocess.py:440]:INFO:Expected error for obtaining projection stats: 3.760816745498263


KeyboardInterrupt: 

In [17]:
stats = pickle.load(open('/data/LiftPose3D/fly_tether/angle_inv_network/stats.pkl','rb'))

In [28]:
lp3d_train(train_2d=train_2d, test_2d=test_2d,
           train_3d=train_3d, test_3d=test_3d, 
           train_keypts=train_keypts,
           test_keypts=test_keypts,
           roots=par['roots'],
           target_sets=par['target_sets'],
           out_dir=par['out_dir'],
           augmentation=[aug],
           mean=stats)

[main.py:240]:INFO:Saving pre-processed 2D data at /data/LiftPose3D/fly_tether/angle_inv_network/stat_2d.pth.tar.
[main.py:259]:INFO:Saving pre-processed 3D data at /data/LiftPose3D/fly_tether/angle_inv_network/stat_3d.pth.tar.
[main.py:287]:INFO:Starting training model.
[main.py:297]:DEBUG:
==================Options=================
[main.py:298]:DEBUG:{   'batch_size': 64,
    'data_dir': '/data/LiftPose3D/fly_tether/angle_inv_network',
    'dropout': 0.5,
    'epochs': 30,
    'exp': '',
    'is_train': True,
    'job': 8,
    'linear_size': 1024,
    'load': None,
    'lr': 0.001,
    'lr_decay': 5000,
    'lr_gamma': 0.9,
    'max_norm': True,
    'noise': None,
    'num_stage': 2,
    'out': '/data/LiftPose3D/fly_tether/angle_inv_network',
    'out_dir': '/data/LiftPose3D/fly_tether/angle_inv_network',
    'predict': False,
    'procrustes': False,
    'resume': False,
    'test': False}
[main.py:299]:DEBUG:==========================================

[lift.py:31]:INFO:Training on

Epoch 000 | LR  0.00100 | Loss Test  0.00000 | Loss Train  0.00000|:   0%|          | 0/3291 [00:00<?, ?it/s]

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000, -1.5429, -0.6528, -5.3006, -1.3123,
        -0.8970, -2.1795, -0.7023,  4.0008,  1.1199,  2.0887,  9.0526, -1.6259,
        -2.7258,  1.7153,  2.3560, -0.5842, -5.7141,  1.9024,  7.0294,  0.1747,
        -0.2390, -3.3905,  8.2684,  9.2919, -2.3028,  1.5776,  0.1127, -1.8239,
        -0.9101,  6.8911, -7.1719,  4.1074, -2.2328,  1.9417, -3.3321,  7.7410],
       device='cuda:0', grad_fn=<SliceBackward>)
tensor([   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
        

SystemExit: 

In [ ]:
%matplotlib inline
from liftpose.plot import read_log_train, plot_log_train
epoch, lr, loss_train, loss_test, err_test = read_log_train(par['out_dir'])
plot_log_train(plt.gca(), loss_train, loss_test, epoch)

# Run Trained LiftPose3D Network on the Test Data

In [ ]:
from liftpose.main import test as lp3d_test
lp3d_test(par['out_dir'])

# Visualize Predictions

In [ ]:
from liftpose.postprocess import load_test_results
data = torch.load(os.path.join(par['out_dir'], "test_results.pth.tar"))
stat_2d, stat_3d = (
    torch.load(os.path.join(par['out_dir'], "stat_2d.pth.tar")),
    torch.load(os.path.join(par['out_dir'], "stat_3d.pth.tar")),
)
test_3d_gt, test_3d_pred, good_keypts = load_test_results(data, stat_2d, stat_3d)

In [ ]:
# https://stackoverflow.com/a/38865534/7554774
# conda install ipympl
#%matplotlib widget
%matplotlib inline
from liftpose.plot import plot_pose_3d

fig = plt.figure(figsize=plt.figaspect(1), dpi=100)
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=90, azim=0)

t = 0
plot_pose_3d(ax=ax, tar=test_3d_gt[t], 
            pred=test_3d_pred[t], 
            bones=par_data["vis"]["bones"], 
            limb_id=par_data["vis"]["limb_id"], 
            colors=par_data["vis"]["colors"],
            good_keypts=good_keypts[t],
            show_gt_always=False)